In [44]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from collections import defaultdict

In [45]:
teamPath = '/Users/justinholmes/Desktop/Sports_Model/Data/teamData.csv'
playerPath = '/Users/justinholmes/Desktop/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/justinholmes/Desktop/Sports_Model/Data/outcomeData.csv'

In [46]:
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [47]:
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

outcomeData.head()

,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner
0,2005,november,Houston Rockets,79,Detroit Pistons,87,NaN,22076,166,Detroit Pistons
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,NaN,20041,205,Dallas Mavericks
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,NaN,18997,167,Los Angeles Lakers
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2OT,19730,213,Indiana Pacers
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,NaN,15138,185,Orlando Magic


In [42]:
outcomeData["OT?"].unique()

array([nan, '2OT', 'OT', '3OT', '4OT'], dtype=object)

In [ ]:
outcomeData.loc[(outcomeData["OT?"] == 'Dance'),'Event']='Hip-Hop'


## Functions for additional calculations

In [48]:
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False

In [49]:
outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [26]:
outcomeData["HomeLastWin"] = False
outcomeData["VisitorLastWin"] = False

In [29]:
teamData.head()

,Team,Year,G_x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,...,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G
0,Phoenix Suns*,2005,82,241.2,40.9,85.6,0.477,9.7,24.7,0.393,...,12.4,27.5,0.222,0.478,12.0,68.3,0.176,America West Arena,"726,066","17,709"
1,Sacramento Kings*,2005,82,242.1,39.1,85.1,0.459,6.4,17.0,0.374,...,12.0,28.8,0.226,0.493,13.3,69.0,0.214,ARCO Arena (II),"709,997","17,317"
2,Dallas Mavericks*,2005,82,240.6,37.3,81.6,0.457,5.6,15.5,0.364,...,12.5,28.6,0.273,0.472,14.3,70.2,0.239,American Airlines Center,"822,533","20,062"
3,Miami Heat*,2005,82,243.7,37.8,77.7,0.486,5.8,15.4,0.377,...,13.1,27.0,0.260,0.460,12.4,74.1,0.242,AmericanAirlines Arena,"815,143","19,882"
4,Boston Celtics*,2005,82,242.4,37.1,79.4,0.468,5.3,15.3,0.349,...,14.7,27.2,0.273,0.481,14.2,70.1,0.274,FleetCenter,"656,081","16,002"


In [28]:
def pointDiff(dataframe):
    
    if teamData['year'] == outcomeData["year"] and teamData['Team'] == outcomeData["Team"]:
        homeScore = teamData[]
    
    return AVGPointDiff

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [50]:
outcomeData.head()

,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner,HomeWin
0,2005,november,Houston Rockets,79,Detroit Pistons,87,NaN,22076,166,Detroit Pistons,True
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,NaN,20041,205,Dallas Mavericks,True
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,NaN,18997,167,Los Angeles Lakers,True
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2OT,19730,213,Indiana Pacers,False
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,NaN,15138,185,Orlando Magic,True


### Calculating a basis level by always predicting the home team will win

In [51]:
# target column (in this case, whether or not the home team will win)
y = outcomeData["HomeWin"].to_numpy()

# columns that will be used to make the prediction
X = outcomeData[["Visitor", "Home"]]

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [52]:
# Set up train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

#### Accuracy of only guessing home team

In [53]:
basis_level = np.count_nonzero(y_train)/len(y_train)
basis_level

0.5951550743255644

#### F1 score of only guessing home team

In [54]:
y_pred = [1] * len(y_train)
f1_score(y_train, y_pred, pos_label = None, average = "weighted")

0.4441067432208479

### Testing with additional variables

In [ ]:
# target column (in this case, whether or not the home team will win)
y = outcomeData["HomeWin"].to_numpy()

# columns that will be used to make the prediction
X = outcomeData[["Visitor", "Home"]]

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [ ]:
# Set up train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [ ]:
clf = RandomForestClassified(random_state = 14)
scores = cross_val_score(clf, X_teams, y_true, scoring=scorer)